In [1]:
import re
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import sqlite3

In [2]:
df_abusive = pd.read_csv("Asset Challenge/abusive.csv",encoding='latin-1')
df_abusive.head()

,ABUSIVE
0,alay
1,ampas
2,buta
3,keparat
4,anjing


In [3]:
df_alay = pd.read_csv("Asset Challenge/new_kamusalay.csv",encoding='latin-1',header=None)
df_alay = df_alay.rename(columns={0:'alay', 1:'tidak_alay'})
df_alay.head()

,alay,tidak_alay
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [4]:
factory = StopWordRemoverFactory()
list_stopword = factory.get_stop_words()
len(list_stopword)

126

In [5]:
for word in list_stopword :
    if word in df_abusive:
        list_stopword.remove(word)

In [6]:
list_stopword

['yang',
 'untuk',
 'pada',
 'ke',
 'para',
 'namun',
 'menurut',
 'antara',
 'dia',
 'dua',
 'ia',
 'seperti',
 'jika',
 'jika',
 'sehingga',
 'kembali',
 'dan',
 'tidak',
 'ini',
 'karena',
 'kepada',
 'oleh',
 'saat',
 'harus',
 'sementara',
 'setelah',
 'belum',
 'kami',
 'sekitar',
 'bagi',
 'serta',
 'di',
 'dari',
 'telah',
 'sebagai',
 'masih',
 'hal',
 'ketika',
 'adalah',
 'itu',
 'dalam',
 'bisa',
 'bahwa',
 'atau',
 'hanya',
 'kita',
 'dengan',
 'akan',
 'juga',
 'ada',
 'mereka',
 'sudah',
 'saya',
 'terhadap',
 'secara',
 'agar',
 'lain',
 'anda',
 'begitu',
 'mengapa',
 'kenapa',
 'yaitu',
 'yakni',
 'daripada',
 'itulah',
 'lagi',
 'maka',
 'tentang',
 'demi',
 'dimana',
 'kemana',
 'pula',
 'sambil',
 'sebelum',
 'sesudah',
 'supaya',
 'guna',
 'kah',
 'pun',
 'sampai',
 'sedangkan',
 'selagi',
 'sementara',
 'tetapi',
 'apakah',
 'kecuali',
 'sebab',
 'selain',
 'seolah',
 'seraya',
 'seterusnya',
 'tanpa',
 'agak',
 'boleh',
 'dapat',
 'dsb',
 'dst',
 'dll',
 'dahulu

In [7]:
list_abusive = df_abusive.values.tolist()
list_kamusalay = list(zip(df_alay['alay'].values.tolist(), df_alay['tidak_alay'].values.tolist()))


In [8]:
conn = sqlite3.connect('challenge_database.db')
cursor = conn.cursor()
# conn.execute("PRAGMA busy_timeout = 30000")

#Membuat abusive_table di db
cursor.execute('''
CREATE TABLE IF NOT EXISTS abusive_table (
    abusive TEXT
    )
''')

cursor.executemany('''
INSERT INTO abusive_table (abusive) VALUES (?)''', list_abusive)

conn.commit()
conn.close()

In [9]:
conn = sqlite3.connect('challenge_database.db')
cursor = conn.cursor()
#Membuat kamus_alay_table di db
cursor.execute('''
CREATE TABLE IF NOT EXISTS kamus_alay_table (
    alay TEXT,
    tidak_alay TEXT
    )
''')

cursor.executemany('''
INSERT INTO kamus_alay_table (alay, tidak_alay) VALUES (?, ?)''', list_kamusalay)
conn.commit()
conn.close()

In [10]:
conn = sqlite3.connect('challenge_database.db')
cursor = conn.cursor()
#Membuat stopword_table di db
cursor.execute('''
CREATE TABLE IF NOT EXISTS stopword_table (
    stopword TEXT
    )
''')
for stopword in list_stopword:
    cursor.execute('''
    INSERT INTO stopword_table (stopword) VALUES (?);''', (stopword,))
conn.commit()
conn.close()

In [11]:
conn = sqlite3.connect('challenge_database.db')
stopword_table = pd.read_sql_query("""
                    SELECT 
                        *
                    FROM stopword_table
                  """, conn)
conn.close()
stopword_table

,stopword
0,yang
1,untuk
2,pada
3,ke
4,para
...,...
121,tolong
122,tentu
123,amat
124,apalagi


In [12]:
conn = sqlite3.connect('challenge_database.db')
abusive_table = pd.read_sql_query("""
                    SELECT 
                        *
                    FROM abusive_table
                  """, conn)
conn.close()
abusive_table

,abusive
0,alay
1,ampas
2,buta
3,keparat
4,anjing
...,...
120,rezim
121,sange
122,serbet
123,sipit


In [13]:
conn = sqlite3.connect('challenge_database.db')
kamus_alay_table = pd.read_sql_query("""
                    SELECT 
                        *
                    FROM kamus_alay_table
                  """, conn)
conn.close()
kamus_alay_table

,alay,tidak_alay
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
...,...,...
15162,mendikbud,menteri pendidikan dan kebudayaan
15163,mendag,menteri perdagangan
15164,menaker,menteri tenaga kerja
15165,memetwit,mentwit
